In [24]:
import pandas as pd

In [25]:
from aicentro.session import Session
session = Session(verify=False)

In [26]:
import tensorflow as tf
import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.optimizers import RMSprop

In [27]:
voc_df = pd.read_csv('/aihub/workspace/AIProject_202007/Data/voc_preScale_final_2_20200721.csv', encoding = 'UTF-8')

In [28]:
del voc_df['Unnamed: 0']

In [29]:
voc_df

,상담대분류,상담중분류,상담소분류,상담일자,상품구분,급지,해지일,AS신청유형,AS원인유형,AS처리유형,AS처리시간,cnslt_rscs_diff,rscs_yn
0,4,10,27,2016-01-01,0,3,2016-06-15,25,86,68,-1.0,166,False
1,4,10,27,2016-01-01,0,4,2015-12-31,25,86,68,-1.0,-1,False
2,4,10,27,2016-01-01,0,2,1999-01-01,25,86,68,-1.0,-6209,False
3,2,11,56,2016-01-01,1,1,1999-01-01,18,75,55,16.0,-6209,False
4,4,10,27,2016-01-02,0,3,1999-01-01,25,86,68,-1.0,-6210,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,2,18,53,2020-07-14,1,4,1999-01-01,25,86,68,-1.0,-7865,False
40484,2,18,40,2020-07-14,1,2,1999-01-01,25,86,68,-1.0,-7865,False
40485,3,6,47,2020-07-14,1,3,1999-01-01,25,86,68,-1.0,-7865,False
40486,2,18,54,2020-07-14,1,6,1999-01-01,25,86,68,-1.0,-7865,False


In [30]:
voc_df.corr()

,상담대분류,상담중분류,상담소분류,상품구분,급지,AS신청유형,AS원인유형,AS처리유형,AS처리시간,cnslt_rscs_diff,rscs_yn
상담대분류,1.000000,-0.613951,-0.064542,-0.858035,-0.049287,0.100360,0.140734,0.132088,-0.050805,0.050805,-0.103012
상담중분류,-0.613951,1.000000,0.104928,0.309086,-0.018732,0.046618,0.075836,0.074865,-0.033488,-0.025873,0.123457
상담소분류,-0.064542,0.104928,1.000000,0.150294,-0.015071,0.088229,0.113209,0.117037,-0.069336,0.087981,0.096098
상품구분,-0.858035,0.309086,0.150294,1.000000,0.086309,-0.118083,-0.162363,-0.155013,0.053537,-0.025106,0.078850
급지,-0.049287,-0.018732,-0.015071,0.086309,1.000000,0.023191,0.014625,0.009234,0.040602,-0.048225,-0.046930
AS신청유형,0.100360,0.046618,0.088229,-0.118083,0.023191,1.000000,0.269823,0.255534,-0.082356,0.050427,0.017585
AS원인유형,0.140734,0.075836,0.113209,-0.162363,0.014625,0.269823,1.000000,0.911809,-0.137373,0.055328,0.015896
AS처리유형,0.132088,0.074865,0.117037,-0.155013,0.009234,0.255534,0.911809,1.000000,-0.126009,0.058493,0.019658
AS처리시간,-0.050805,-0.033488,-0.069336,0.053537,0.040602,-0.082356,-0.137373,-0.126009,1.000000,-0.022638,-0.016578
cnslt_rscs_diff,0.050805,-0.025873,0.087981,-0.025106,-0.048225,0.050427,0.055328,0.058493,-0.022638,1.000000,0.433840


In [31]:
val_columns = ['상담대분류', '상담중분류', '상담소분류', '급지', '상품구분', 'AS신청유형', 'AS원인유형', 'AS처리유형', 'rscs_yn']
voc_df = pd.DataFrame(voc_df, columns=val_columns)

In [32]:
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,4,10,27,3,0,25,86,68,False
1,4,10,27,4,0,25,86,68,False
2,4,10,27,2,0,25,86,68,False
3,2,11,56,1,1,18,75,55,False
4,4,10,27,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,2,18,53,4,1,25,86,68,False
40484,2,18,40,2,1,25,86,68,False
40485,3,6,47,3,1,25,86,68,False
40486,2,18,54,6,1,25,86,68,False


In [33]:
cnslt_big = [
'(인)고객불만'
,'가입해지'
,'고객불만'
,'고객불만 이관'
,'서비스 문의 및 신청'
,'품질'
,'회사 관련 문의'
]
toCharCoder_cnslt_big={i:k for i,k in enumerate(cnslt_big)}


In [34]:
l=[toCharCoder_cnslt_big[i] for i in voc_df['상담대분류']]
s=pd.Series(l)
voc_df['상담대분류'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,10,27,3,0,25,86,68,False
1,서비스 문의 및 신청,10,27,4,0,25,86,68,False
2,서비스 문의 및 신청,10,27,2,0,25,86,68,False
3,고객불만,11,56,1,1,18,75,55,False
4,서비스 문의 및 신청,10,27,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,18,53,4,1,25,86,68,False
40484,고객불만,18,40,2,1,25,86,68,False
40485,고객불만 이관,6,47,3,1,25,86,68,False
40486,고객불만,18,54,6,1,25,86,68,False


In [35]:
cnslt_mid = [
'AS처리점 유발'
,'CRG호전환'
,'고객센터/유통망관련문의'
,'과기정통부'
,'기타 단체'
,'내방고객'
,'대한주부클럽연합회'
,'방송통신위원회'
,'본사 전화민원'
,'서비스 문의 및 신청'
,'서비스 신청'
,'설치(이전)/AS'
,'소비자보호원'
,'소비자신문'
,'소비자연맹'
,'수신기'
,'수신장애'
,'시민의모임'
,'영업'
]

toCharCoder_cnslt_mid={i:k for i,k in enumerate(cnslt_mid)}

In [36]:
l=[toCharCoder_cnslt_mid[i] for i in voc_df['상담중분류']]
s=pd.Series(l)
voc_df['상담중분류'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,27,3,0,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,27,4,0,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,27,2,0,25,86,68,False
3,고객불만,설치(이전)/AS,56,1,1,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,27,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,53,4,1,25,86,68,False
40484,고객불만,영업,40,2,1,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,47,3,1,25,86,68,False
40486,고객불만,영업,54,6,1,25,86,68,False


In [37]:
cnslt_small = [
'(AS기사) 방문약속 미이행'
,'(AS기사) 방문약속 미이행'
,'(AS기사) 불친절'
,'(AS기사) 처리지연'
,'(AS기사) 후처리 미흡'
,'(문의) 시청불가'
,'(문의) 신호미약'
,'(설치기사) 방문약속 미이행'
,'(설치기사) 불친절'
,'(설치기사) 처리지연'
,'(설치기사) 후처리 미흡'
,'(접수) 기타메시지'
,'(접수) 리모콘 동작불가'
,'(접수) 수신기 동작불가'
,'(접수) 수신설비 파손 분실'
,'(접수) 신호미약'
,'(접수) 영상불량'
,'(접수) 음성불량'
,'(접수)스마트카드 오류 메시지'
,'AS 중복발생'
,'AS비용불만(즉납)'
,'AS점 직원 불친절'
,'AS점불만'
,'AS지연'
,'AS출동지연(도서지역)'
,'AS출동지연(약속미이행)'
,'AS출동지연(중복)SMS'
,'SkyChoice'
,'가격정책(제도)'
,'가전결합 불만'
,'개인정보 침해'
,'고객재산 파손 및 손괴'
,'기사칭찬'
,'기업 상품'
,'기타 단체'
,'기타(특이사례)'
,'명의도용'
,'방송품질'
,'사은품 미지금(약속미준수)'
,'상담'
,'상품/약정안내 미흡'
,'설치(이전)/AS'
,'설치점 불만'
,'설치점불만'
,'설치지연'
,'쌍방향 서비스'
,'업셀링 요금/약정 안내 미흡'
,'영업'
,'영업점불만'
,'영업점불만(신청서 대필)'
,'영업점불만(오처리/처리누락)'
,'영업점불만(임의개통_가개통)'
,'영업점불만(임의개통_명의도용)'
,'영업점불만(임의개통_이중가입)'
,'영업점불만(임의개통_일방적가입)'
,'이용불편'
,'잦은 AS'
,'청구/환불'
,'콘텐츠'
,'클레임처리지연'
,'패키지 체계 및 요금'
,'패키지 추가'
,'해지'
,'해지 접수'
]

toCharCoder_cnslt_small={i:k for i,k in enumerate(cnslt_small)}

In [38]:
l=[toCharCoder_cnslt_small[i] for i in voc_df['상담소분류']]
s=pd.Series(l)
voc_df['상담소분류'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,3,0,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,4,0,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,2,0,25,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,1,1,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),4,1,25,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,2,1,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,3,1,25,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),6,1,25,86,68,False


In [39]:
prdt_type = [
'없음'
,'위성'
,'인터넷'
]

toCharCoder_prdt_type={i:k for i,k in enumerate(prdt_type)}

In [40]:
l=[toCharCoder_prdt_type[i] for i in voc_df['상품구분']]
s=pd.Series(l)
voc_df['상품구분'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,2,없음,25,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,1,위성,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),4,위성,25,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,2,위성,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,3,위성,25,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),6,위성,25,86,68,False


In [41]:
region_lvl = [
'1등급'
,'2등급'
,'3등급'
,'4등급'
,'5등급'
,'6등급'
,'7등급'
,'8등급'
,'급지없음'
]

toCharCoder_region_lvl={i:k for i,k in enumerate(region_lvl)}

In [131]:
l=[toCharCoder_region_lvl[i] for i in voc_df['급지']]
s=pd.Series(l)
voc_df['급지'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5등급,없음,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,3등급,없음,25,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,2등급,위성,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),5등급,위성,25,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,3등급,위성,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,4등급,위성,25,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),7등급,위성,25,86,68,False


In [42]:
as_apply = [
'B/S점검'
,'IP백업고객 B/S점검'
,'IP연결요청'
,'PPV 오류 메시지'
,'PVR 녹화/시청 불가'
,'TQC'
,'USB 플레이어/타임머신 동작 불가'
,'VOD 시청 불가'
,'Wifi 사용불가'
,'Wifi 연결안됨'
,'[휴일]수신기 전원 미작동'
,'기타'
,'기타'
,'기타 메시지'
,'네트워크 연결확인 메시지'
,'리모콘 동작불가'
,'리모콘 사용법 모름'
,'모뎀 전원 미동작'
,'수신기 동작불가'
,'수신기 업그레이드안됨'
,'수신기 전원 미동작'
,'수신설비 파손 분실'
,'스마트카드 오류 메시지'
,'신호미약 메시지'
,'실버리모콘'
,'없음'
,'영상 불량'
,'영상 안아놈'
,'음성 불량'
,'음성 안나옴'
,'인터넷 사용불가'
,'재해지역AS발생'
,'케이블 연결방법 모름'
]

toCharCoder_as_apply={i:k for i,k in enumerate(as_apply)}

In [43]:
l=[toCharCoder_as_apply[i] for i in voc_df['AS신청유형']]
s=pd.Series(l)
voc_df['AS신청유형'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,없음,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,없음,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,2,없음,없음,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,1,위성,수신기 동작불가,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,없음,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),4,위성,없음,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,2,위성,없음,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,3,위성,없음,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),6,위성,없음,86,68,False


In [44]:
as_cause = [
'A/S취소'
,'A/V케이블 분리/단선'
,'A/V케이블 불량'
,'A/V케이블 오연결'
,'AS관련 클레임'
,'B/S점검'
,'IP연결요청'
,'LNB 동작불가'
,'LNB 파손'
,'LNB 파손/분실.동작불가'
,'LNB불량'
,'LTE모뎀 동작불가'
,'TP유니트 불량'
,'UTP 케이블 분리/단선'
,'UTP커넥터불량'
,'[FTTH] 포트불량'
,'[FTTH] 환경설정오류'
,'[IP공유기] 고객부주의(사용미숙)'
,'[IP공유기] 접속불량'
,'[IP공유기] 제품불량(제조사)'
,'[NE] 네트워크장비장애'
,'[Ntopia] 불복구'
,'[Ntopia] 포트불량'
,'[TOK] 비고장'
,'[고객취소] 비고장'
,'[구내단자함] 접속불량'
,'[기술상담] 문의성'
,'[기타] 문의성'
,'[기타] 비고장'
,'[모뎀] 고객부주의(사용미숙)'
,'[모뎀] 분실'
,'[모뎀] 불복구'
,'[모뎀] 속도저하'
,'[모뎀] 장애/공사/피해'
,'[모뎀] 접속불량'
,'[모뎀] 제품불량(제조사)'
,'[모뎀] 환경설정오류'
,'[세대단자함] 공법불량'
,'[세대단자함] 접속불량'
,'[실내선] 고객부주의(사용미숙)'
,'[실내선] 장애/공사/피해'
,'[실내선] 접속불량'
,'[인입광] 고객요청(민원)'
,'[인입광] 공법불량'
,'[인입광] 시설노후'
,'[인입광] 장애/공사/피해'
,'[인입광] 접속불량'
,'[인입광] 고객부주의(사용미숙)'
,'[인입선] 시설노후'
,'[인입선] 절연불량'
,'[인입선] 접속불량'
,'[자연회복] 자연회복'
,'[콘넥터] 접속불량'
,'[폐문] 비고장'
,'가스배관 등 위험지역 설치'
,'공유기 사용 미숙지'
,'공유기/허브 동작 불가'
,'기타'
,'녹화방법 미숙지'
,'도시가스배관정비'
,'동작 불가'
,'동작불가'
,'동툭케이블 분리/단선'
,'동축콘넥터 접불'
,'리모콘 동작불가'
,'리모콘 분실'
,'리모콘 추가'
,'리모콘 파손'
,'리모콘사용법 미숙지'
,'방수처리 불량'
,'불량'
,'설치관련 클레임'
,'수신기 LNB전원공급 불가'
,'수신기 동작불가'
,'수신기 분실'
,'수신기 전원 동작불가'
,'수신기 파손'
,'수신기 파손/분식/동작불가'
,'스마트카드 동작불가'
,'스마트카드 분실'
,'신호 불량'
,'안테나 고장불량(앙카볼트 미사용 등)'
,'안테나 동갖불가'
,'안테나 방향 틀어짐'
,'안테나 방향 틀어짐(고객사유)'
,'안테나 파손/분실'
,'없음'
,'일시적 장애'
,'잭 접불'
,'저잡음증폭기 불량'
,'전산 작업 오류'
,'전원'
,'회선 분리/단선'
,'접속불량'
,'정상확인'
,'지상파HD채널설정방법 미숙지'
,'자상파전환방법 미숙지'
,'취약지역 안테나 설치'
,'케이블연결방법 미숙지'
,'타 가입자 안테나 분배 설치'
,'파손/분실/동작불가'
,'해상도 변경 미숙지'
]

toCharCoder_as_cause={i:k for i,k in enumerate(as_cause)}

In [45]:
l=[toCharCoder_as_cause[i] for i in voc_df['AS원인유형']]
s=pd.Series(l)
voc_df['AS원인유형'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,없음,없음,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,없음,없음,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,2,없음,없음,없음,68,False
3,고객불만,설치(이전)/AS,잦은 AS,1,위성,수신기 동작불가,수신기 전원 동작불가,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,없음,없음,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),4,위성,없음,없음,68,False
40484,고객불만,영업,상품/약정안내 미흡,2,위성,없음,없음,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,3,위성,없음,없음,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),6,위성,없음,없음,68,False


In [46]:
as_process = [
'(외부자산)시설교체 및 수리 안내'
,'A/S취소'
,'A/V케이블 교체'
,'A/V케이블 재연결'
,'AS관련 클레임 처리'
,'AS취소'
,'IP연결요청'
,'LNB교체'
,'LNB교체'
,'LTE모뎀 교체'
,'Modulator 교체'
,'UTP 케이블 포설'
,'UTP커넥터 접속'
,'[FTTH] 포트교체'
,'[FTTH] 프로파일재설정'
,'[IP공유기] 교체(모뎀/ONT/STB)'
,'[IP공유기] 수리의뢰(고객)'
,'[IP공유기] 환경재설정'
,'[NE] 무출동네트워크장비복구'
,'[Ntopia] 포트교체'
,'[Ntopia] 프로파일재설정'
,'[구내단자함] 재접속'
,'[모뎀] 교체(모뎀/ONT/STB)'
,'[모뎀] 불복구제거(RESET)'
,'[모뎀] 재접속'
,'[모뎀] 환경재설정'
,'[세대단자함] 선로재구성(인입선 등)'
,'[세대단자함] 재접속'
,'[실내선]LAB케이블 교체'
,'[실내선]선로재구성(인입선 등)'
,'[실내선]재접속'
,'[인입광] 광콘넥터재접속'
,'[인입광] 교체(모뎀/ONT/STB)'
,'[인입광] 선로재구성(인입성 등)'
,'[인입광] 회선정비'
,'[인입선] 선로재구성(인입선 등)'
,'[인입선] 재접속'
,'[콘넥터] 광콘넥터재접속'
,'[콘넥터] 재접속'
,'공유기 사용안내'
,'공유기/허브 전원 재연결'
,'교체'
,'교체-IF증폭기-기타(미승인)'
,'교체-IF증폭기-아진'
,'교체-IF증폭기-청화'
,'교체/수리 안내'
,'기 타'
,'기술상담'
,'기타'
,'녹화방법안내'
,'동축케이블 포설'
,'동축콘넥터 접속'
,'리모콘 교체'
,'리모콘 추가'
,'리모콘사용법 안내'
,'무상수리교체(무료보증기간)'
,'방수(실리콘)처리'
,'배선 등 현장점검'
,'설치관련 클레임 처리'
,'수신기 교체'
,'수신기 전원 재연결'
,'스마트카드 교체'
,'안테나 교체'
,'안테나 교체/설치'
,'안테나 방향 조정'
,'안테나 재설치 또는 점검'
,'안테나재설치'
,'잉카볼트사용 및 고정상태'
,'없음'
,'유니트 교체'
,'유상구매교체(수리불가)'
,'유상수리교체(수리후재설치)'
,'자가보유교체(고객보유분)'
,'자동복구'
,'자연회복'
,'저잡음증폭기교체'
,'전산 작업'
,'전산작업'
,'전원 재입전'
,'전화선 연결'
,'정상확인/종료'
,'조치없음'
,'증폭기 신호조정'
,'증폭기전원재연결'
,'증폭기 추가(지사통보)'
,'지상파HD채널설정방법 안내'
,'지상파 전환방법 안내'
,'커넥터 재작업(재접속)'
,'케이블 연결방법 안내'
,'케이블 재포설(교체)'
,'타센터 AS처리'
,'해상도 변경 안내'
,'현장수리완료(OTA전송)']

toCharCoder_as_process={i:k for i,k in enumerate(as_process)}

In [47]:
l=[toCharCoder_as_cause[i] for i in voc_df['AS처리유형']]
s=pd.Series(l)
voc_df['AS처리유형'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,없음,없음,리모콘사용법 미숙지,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,없음,없음,리모콘사용법 미숙지,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,2,없음,없음,없음,리모콘사용법 미숙지,False
3,고객불만,설치(이전)/AS,잦은 AS,1,위성,수신기 동작불가,수신기 전원 동작불가,공유기 사용 미숙지,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,없음,없음,리모콘사용법 미숙지,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),4,위성,없음,없음,리모콘사용법 미숙지,False
40484,고객불만,영업,상품/약정안내 미흡,2,위성,없음,없음,리모콘사용법 미숙지,False
40485,고객불만 이관,대한주부클럽연합회,영업,3,위성,없음,없음,리모콘사용법 미숙지,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),6,위성,없음,없음,리모콘사용법 미숙지,False


In [48]:
# # 데이터를 읽고 순서를 섞는다
voc_model_df=voc_df.sample(frac=1).reset_index(drop=True)
voc_model_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,1,없음,없음,없음,리모콘사용법 미숙지,False
1,고객불만 이관,소비자연맹,해지,5,위성,없음,없음,리모콘사용법 미숙지,False
2,고객불만,설치(이전)/AS,설치점불만,1,위성,신호미약 메시지,동축콘넥터 접불,[자연회복] 자연회복,False
3,서비스 문의 및 신청,서비스 신청,SkyChoice,2,없음,없음,없음,리모콘사용법 미숙지,False
4,고객불만,설치(이전)/AS,(AS기사) 후처리 미흡,3,위성,영상 불량,안테나 방향 틀어짐,리모콘 동작불가,False
...,...,...,...,...,...,...,...,...,...
40483,서비스 문의 및 신청,서비스 신청,패키지 추가,1,없음,없음,없음,리모콘사용법 미숙지,False
40484,고객불만,설치(이전)/AS,잦은 AS,5,위성,기타 메시지,수신기 동작불가,동작 불가,False
40485,고객불만,영업,영업점불만(오처리/처리누락),1,위성,없음,없음,리모콘사용법 미숙지,False
40486,고객불만,설치(이전)/AS,(설치기사) 방문약속 미이행,2,위성,신호미약 메시지,리모콘사용법 미숙지,가스배관 등 위험지역 설치,False


In [49]:
voc_model_df.dtypes

상담대분류      object
상담중분류      object
상담소분류      object
급지          int64
상품구분       object
AS신청유형     object
AS원인유형     object
AS처리유형     object
rscs_yn      bool
dtype: object

In [50]:
!pip install JPype1
!pip install konlpy

Looking in indexes: http://aicentro-nexus-svc:8081/hub-nexus/repository/pypi-group/simple
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://aicentro-nexus-svc:8081/hub-nexus/repository/pypi-group/simple
You should consider upgrading via the 'pip install --upgrade pip' command.


In [53]:
from konlpy.tag import Kkma
kkma = Kkma()

In [54]:
cnslt_dic = print(kkma.nouns(u'고객불만 이관'))

['고객', '고객불만', '불만', '이관']


In [55]:
voc_df["상담"] = voc_df["상담대분류"] + voc_df["상담중분류"] + voc_df["상담소분류"]

In [56]:
voc_df['AS'] = voc_df["AS신청유형"] + voc_df['AS원인유형'] + ['AS처리유형']

In [86]:
voc_df['상담+AS'] = voc_df['상담'] + voc_df['AS']

In [87]:
val_columns = ['상담+AS','상담', 'AS', '급지', '상품구분', 'rscs_yn']
voc_df = pd.DataFrame(voc_df, columns=val_columns)

In [88]:
voc_df

,상담+AS,상담,AS,급지,상품구분,rscs_yn
0,서비스 문의 및 신청서비스 신청SkyChoice없음없음AS처리유형,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,3,없음,False
1,서비스 문의 및 신청서비스 신청SkyChoice없음없음AS처리유형,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,4,없음,False
2,서비스 문의 및 신청서비스 신청SkyChoice없음없음AS처리유형,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,2,없음,False
3,고객불만설치(이전)/AS잦은 AS수신기 동작불가수신기 전원 동작불가AS처리유형,고객불만설치(이전)/AS잦은 AS,수신기 동작불가수신기 전원 동작불가AS처리유형,1,위성,False
4,서비스 문의 및 신청서비스 신청SkyChoice없음없음AS처리유형,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,3,없음,False
...,...,...,...,...,...,...
40483,고객불만영업영업점불만(임의개통_이중가입)없음없음AS처리유형,고객불만영업영업점불만(임의개통_이중가입),없음없음AS처리유형,4,위성,False
40484,고객불만영업상품/약정안내 미흡없음없음AS처리유형,고객불만영업상품/약정안내 미흡,없음없음AS처리유형,2,위성,False
40485,고객불만 이관대한주부클럽연합회영업없음없음AS처리유형,고객불만 이관대한주부클럽연합회영업,없음없음AS처리유형,3,위성,False
40486,고객불만영업영업점불만(임의개통_일방적가입)없음없음AS처리유형,고객불만영업영업점불만(임의개통_일방적가입),없음없음AS처리유형,6,위성,False


In [90]:
nlp_dic = []
#ll = [kkma.nouns(i) for i in voc_df['상담'].head(1000)]
def add_dic(s):
    global nlp_dic
    nlp_dic=list(set(nlp_dic+list(s)))
#     print(nlp_dic)
for i in voc_df['상담+AS']:
    ii=set(kkma.nouns(i))
    add_dic(ii)
    
print('---')    
print(nlp_dic)

---
['클레임', '칭찬기타동', '환경', '동축콘넥터', '환불', '발생수신기', '속도저하', '휴일', '중복발생수신기', '침해', '메시지일시적', '환불리모콘', '이관내방고객설치', '오연결', '회사', '단체방송품질', '불친절기타', '이행', '장비', '이관소비자보호원영업', '동작불가잭', '연결', '파전환방법', '공', '이관기타', '전화민원방송품질', '불', '기타동', '불량자상파전환방법', '기정통부해지', '유통망관련', '이관소비자신문기타', '스마트', '미흡영상', '미이행기타', '영업점불만', '구내', '이관소비자보호원해지', '연맹', '지연영상', '시청불가', '주부클럽연합회', '절', '단체영상', '청구', '신호미약', '환불신호미약', '전화민원영업영상', '모임명의도용', '통신', '녹화', '미동작리모콘사용법', '메시지동작불가', '연결요청', '미이행', '수신', '제도', '민원', '위험지역', '미이행영상', '메시지리모콘', '임의개통', '모임방송', '연결요청리모콘사용법', '메시지불량', '점불만', '영상불량', '단체이용불편', '비고장', '정책', '불만기타', '신청패키지', '단체방송품질수신기', '해지', '미지금', '기정', '임의', '중복발생신호미약', '점불만재해지역', '점불만수신기', '미동작리모콘', '시청불가음성', '단체재해지역', '전화민원상담음성', '자연', '등', '모름회선', '영상불량기타리모콘', '정오', '분실스마트카드', '후', '보호원이용', '타스', '커넥터', '상담', '미흡기타', '방송품질수신기', '분실동축콘넥터', '가스', '구내단자함', '콘텐츠리모콘', '주의', '방문약속', '산', '지연', '이관소비자보호원청구', '고객부주의', '모름케이블연결방법', '메시지안테나', '메시지수신기', '이관대', '이관소비자신문영업', '영상', '불친절', '미동작동', '일방적가입', '단체기타동'

In [93]:
import pickle

In [94]:
 with open('nlp_dic.pkl', 'wb') as f:
        pickle.dump(nlp_dic, f)

In [166]:
nlp_dic

['클레임',
 '칭찬기타동',
 '환경',
 '동축콘넥터',
 '환불',
 '발생수신기',
 '속도저하',
 '휴일',
 '중복발생수신기',
 '침해',
 '메시지일시적',
 '환불리모콘',
 '이관내방고객설치',
 '오연결',
 '회사',
 '단체방송품질',
 '불친절기타',
 '이행',
 '장비',
 '이관소비자보호원영업',
 '동작불가잭',
 '연결',
 '파전환방법',
 '공',
 '이관기타',
 '전화민원방송품질',
 '불',
 '기타동',
 '불량자상파전환방법',
 '기정통부해지',
 '유통망관련',
 '이관소비자신문기타',
 '스마트',
 '미흡영상',
 '미이행기타',
 '영업점불만',
 '구내',
 '이관소비자보호원해지',
 '연맹',
 '지연영상',
 '시청불가',
 '주부클럽연합회',
 '절',
 '단체영상',
 '청구',
 '신호미약',
 '환불신호미약',
 '전화민원영업영상',
 '모임명의도용',
 '통신',
 '녹화',
 '미동작리모콘사용법',
 '메시지동작불가',
 '연결요청',
 '미이행',
 '수신',
 '제도',
 '민원',
 '위험지역',
 '미이행영상',
 '메시지리모콘',
 '임의개통',
 '모임방송',
 '연결요청리모콘사용법',
 '메시지불량',
 '점불만',
 '영상불량',
 '단체이용불편',
 '비고장',
 '정책',
 '불만기타',
 '신청패키지',
 '단체방송품질수신기',
 '해지',
 '미지금',
 '기정',
 '임의',
 '중복발생신호미약',
 '점불만재해지역',
 '점불만수신기',
 '미동작리모콘',
 '시청불가음성',
 '단체재해지역',
 '전화민원상담음성',
 '자연',
 '등',
 '모름회선',
 '영상불량기타리모콘',
 '정오',
 '분실스마트카드',
 '후',
 '보호원이용',
 '타스',
 '커넥터',
 '상담',
 '미흡기타',
 '방송품질수신기',
 '분실동축콘넥터',
 '가스',
 '구내단자함',
 '콘텐츠리모콘',
 '주의',
 '방문약속',
 '산',
 '지연',
 '이관소비자보호원청구'

In [173]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [174]:
vector = CountVectorizer()
vector.fit_transform(nlp_dic)

<841x809 sparse matrix of type '<class 'numpy.int64'>'
	with 809 stored elements in Compressed Sparse Row format>

In [175]:
cnslt_array = vector.transform(voc_df['상담']).toarray()

cnslt_array.shape

(40488, 809)

In [176]:
vector.vocabulary_

{'클레임': 751,
 '칭찬기타동': 736,
 '환경': 787,
 '동축콘넥터': 155,
 '환불': 789,
 '발생수신기': 293,
 '속도저하': 446,
 '휴일': 808,
 '중복발생수신기': 685,
 '침해': 732,
 '메시지일시적': 178,
 '환불리모콘': 795,
 '이관내방고객설치': 545,
 '오연결': 526,
 '회사': 805,
 '단체방송품질': 95,
 '불친절기타': 381,
 '이행': 575,
 '장비': 593,
 '이관소비자보호원영업': 553,
 '동작불가잭': 150,
 '연결': 492,
 '파전환방법': 761,
 '이관기타': 543,
 '전화민원방송품질': 605,
 '기타동': 48,
 '불량자상파전환방법': 357,
 '기정통부해지': 43,
 '유통망관련': 538,
 '이관소비자신문기타': 556,
 '스마트': 455,
 '미흡영상': 281,
 '미이행기타': 244,
 '영업점불만': 521,
 '구내': 25,
 '이관소비자보호원해지': 555,
 '연맹': 499,
 '지연영상': 699,
 '시청불가': 461,
 '주부클럽연합회': 669,
 '단체영상': 106,
 '청구': 724,
 '신호미약': 473,
 '환불신호미약': 799,
 '전화민원영업영상': 614,
 '모임명의도용': 207,
 '통신': 757,
 '녹화': 62,
 '미동작리모콘사용법': 230,
 '메시지동작불가': 168,
 '연결요청': 495,
 '미이행': 243,
 '수신': 451,
 '제도': 663,
 '민원': 286,
 '위험지역': 532,
 '미이행영상': 258,
 '메시지리모콘': 170,
 '임의개통': 582,
 '모임방송': 208,
 '연결요청리모콘사용법': 496,
 '메시지불량': 172,
 '점불만': 631,
 '영상불량': 503,
 '단체이용불편': 115,
 '비고장': 406,
 '정책': 661,
 '불만기타': 365,
 '신청패키지': 470,

In [177]:
d=vector.vocabulary_

In [227]:
# cnslt_df = pd.DataFrame(cnslt_array,columns=d.keys())

cnslt_df = pd.DataFrame(cnslt_array)

In [228]:
as_array = vector.transform(voc_df['AS']).toarray()

as_array.shape

(40488, 809)

In [229]:
as_df = pd.DataFrame(as_array) #,columns=d.keys())

as_df

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [230]:
val_nlp_columns = ['급지', '상품구분', 'rscs_yn']
voc_nlp_df = pd.DataFrame(voc_df, columns=val_nlp_columns)
voc_nlp_df.columns=['gupji','prdttype','rscs_yn']

In [231]:
# 속도 개선을 위해 파이프라인을 사용하도록 개선
# 참고 : https://stackoverflow.com/questions/28160335/plot-a-document-tfidf-2d-graph
pipeline = Pipeline([
    ('vect', vectorizer),
]) 

In [232]:
as_df

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [253]:
as_df.columns=[100000+i for i in as_df.columns]

In [254]:
as_df

,100000,100001,100002,100003,100004,100005,100006,100007,100008,100009,...,100799,100800,100801,100802,100803,100804,100805,100806,100807,100808
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [256]:
cnslt_df

,0,1,2,3,4,5,6,7,8,9,...,799,800,801,802,803,804,805,806,807,808
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [257]:
voc_nlp_df

,voc_nlp_gupji,voc_nlp_prdttype,voc_nlp_rscs_yn
0,3,없음,False
1,4,없음,False
2,2,없음,False
3,1,위성,False
4,3,없음,False
...,...,...,...
40483,4,위성,False
40484,2,위성,False
40485,3,위성,False
40486,6,위성,False


In [286]:
#pd.concat(voc_nlp_df,as_df)
voc_final_df = pd.concat([as_df, cnslt_df,voc_nlp_df], axis=1)
del voc_final_df['voc_nlp_prdttype']
voc_final_df

,100000,100001,100002,100003,100004,100005,100006,100007,100008,100009,...,801,802,803,804,805,806,807,808,voc_nlp_gupji,voc_nlp_rscs_yn
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,False
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,False
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,False
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,False
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,False
40484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,False
40485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,False
40486,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,False


In [287]:
voc_train=voc_final_df.iloc[0:20000,:]
voc_test=voc_final_df.iloc[20000:40488,:]

In [288]:
x_train= voc_train.iloc[:,0:1619].values
x_test= voc_test.iloc[:,0:1619].values
y_train= voc_train.iloc[:,1619:1620]
y_test= voc_test.iloc[:,1619:1620]

In [289]:
y_train

,voc_nlp_rscs_yn
0,False
1,False
2,False
3,False
4,False
...,...
19995,True
19996,False
19997,False
19998,False


In [290]:
# encoder={k:v for v,k in enumerate(y_train.drop_duplicates())}
# encoder
sets=voc_final_df.iloc[:,1619:1620].drop_duplicates()["voc_nlp_rscs_yn"].tolist()

In [291]:
encoder={k:v for v,k in enumerate(sets)}

In [292]:
y_train=[ encoder[i] for i in y_train["voc_nlp_rscs_yn"].tolist() ]

In [293]:
batch_size = 50
num_classes = 3
epochs = 200

In [294]:
y_train = keras.utils.to_categorical(y_train, num_classes)

y_test=[ encoder[i] for i in y_test["voc_nlp_rscs_yn"].tolist() ]
y_test = keras.utils.to_categorical(y_test, num_classes)

In [295]:
y_test

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [296]:
# 데이터 shape 을 확인해 봅니다.
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(20000, 1619) (20000, 3) (20488, 1619) (20488, 3)


In [297]:
model = Sequential()
model.add(Dense(18, activation='relu', input_shape=(1619,)))
model.add(Dropout(0.4))
model.add(Dense(20, activation='tanh',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.4))
model.add(Dense(5, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))

model.summary()
#'binary_crossentropy'
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 18)                29160     
_________________________________________________________________
dropout_3 (Dropout)          (None, 18)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)                380       
_________________________________________________________________
dropout_4 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 105       
_________________________________________________________________
dropout_5 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                

In [298]:
import  tensorflow.keras.optimizers
dir( tensorflow.keras.optimizers)

['Adadelta',
 'Adagrad',
 'Adam',
 'Adamax',
 'Ftrl',
 'Nadam',
 'Optimizer',
 'RMSprop',
 'SGD',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_deprecation_wrapper',
 '_sys',
 'deserialize',
 'get',
 'schedules',
 'serialize']

In [299]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 20000 samples, validate on 20488 samples
Epoch 1/200
20000/20000 [==============================] - 3s 163us/sample - loss: 0.4463 - acc: 0.8775 - val_loss: 0.3270 - val_acc: 0.8882
Epoch 2/200
20000/20000 [==============================] - 3s 141us/sample - loss: 0.2953 - acc: 0.9039 - val_loss: 0.3084 - val_acc: 0.8882
Epoch 3/200
20000/20000 [==============================] - 3s 129us/sample - loss: 0.2829 - acc: 0.9043 - val_loss: 0.3047 - val_acc: 0.8882
Epoch 4/200
20000/20000 [==============================] - 3s 132us/sample - loss: 0.2792 - acc: 0.9043 - val_loss: 0.3028 - val_acc: 0.8882
Epoch 5/200
20000/20000 [==============================] - 3s 137us/sample - loss: 0.2762 - acc: 0.9043 - val_loss: 0.3014 - val_acc: 0.8882
Epoch 6/200
20000/20000 [==============================] - 3s 130us/sample - loss: 0.2758 - acc: 0.9043 - val_loss: 0.3022 - val_acc: 0.8882
Epoch 7/200
20000/20000 [==============================] - 3s 134us/sample - loss: 0.2748 - acc: 0.9043 

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])